In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

In [0]:
infer_data_path = dbutils.widgets.get("infer_path")
infer_data_path

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2018934100508897> in <cell line: 1>()
----> 1 infer_data_path = dbutils.widgets.get("path")
      2 infer_data_path

/databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get(self, name)
     40         :return: Current value of the widget or default value
     41         """
---> 42         return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())
     43 
     44     def getArgument(self, name, defaultValue=None):

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1319 
   1320         answer = self.gateway_client.send_command(command)
-> 1321         return_value = get_return_value(
   1322             answer, self.gateway_client, self.target_id, self.name)
   1323 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    194   

In [0]:
from pyspark.ml.classification import LogisticRegressionModel
persistedModel = LogisticRegressionModel.load('dbfs:/FileStore/diabetes/model')

In [0]:
#dbutils.fs.ls('FileStore/diabetes/diabetes_test.csv')

Out[32]: [FileInfo(path='dbfs:/FileStore/diabetes/diabetes_test.csv', name='diabetes_test.csv', size=2180, modificationTime=1677539632000)]

In [0]:
df_infer = spark.read.csv(path=infer_data_path, header=True, inferSchema=True)
#"dbfs:/FileStore/diabetes/diabetes_test.csv"

In [0]:
df_infer.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    125|           78|           31|      0|27.6|                   0.565| 49|      1|
|          1|    168|           88|           29|      0|35.0|                   0.905| 52|      1|
|          2|    129|            0|            0|      0|38.5|                   0.304| 41|      0|
|          4|    110|           76|           20|    100|28.4|                   0.118| 27|      0|
|          6|     80|           80|           36|      0|39.8|                   0.177| 28|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import col,isnan, when, count
df_infer.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_infer.columns]
   ).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler().setInputCols(['Pregnancies','Glucose','BloodPressure','SkinThickness',\
                                            'Insulin','BMI','DiabetesPedigreeFunction','Age']).setOutputCol("features")

In [0]:
df_infer_processed = assembler.transform(df_infer)

In [0]:
df_infer_processed = df_infer_processed.select("features")

In [0]:
preds = persistedModel.transform(df_infer_processed)

In [0]:
preds.show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[6.0,125.0,78.0,3...|[0.58236411369926...|[0.64161120725542...|       0.0|
|[1.0,168.0,88.0,2...|[-1.0873225706602...|[0.25212279100730...|       1.0|
|[2.0,129.0,0.0,0....|[-0.4684911004929...|[0.38497344200727...|       1.0|
|[4.0,110.0,76.0,2...|[2.03732981525018...|[0.88466109311825...|       0.0|
|[6.0,80.0,80.0,36...|[1.52515751046992...|[0.82129669507605...|       0.0|
|(8,[0,1,6,7],[10....|[2.78156850874846...|[0.94167165630823...|       0.0|
|[2.0,127.0,46.0,2...|[1.08955464446249...|[0.74829784897901...|       0.0|
|[9.0,164.0,78.0,0...|[-0.9969013964869...|[0.26955107979963...|       1.0|
|[2.0,93.0,64.0,32...|[1.42368704194355...|[0.80591577761965...|       0.0|
|[3.0,158.0,64.0,1...|[0.41937563674091...|[0.60333383557163...|       0.0|
|[5.0,126.0,

In [0]:
preds.write.format("parquet").mode("overwrite").save('dbfs:/FileStore/diabetes/output')